Load the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installation gor Gen model

In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.

importing Libraries


In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Import the document

In [ ]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Data")
documents = loader.load()

In [ ]:
len(documents) #number of pages

95

In [ ]:
documents[4]

Document(metadata={'source': '/content/drive/MyDrive/Data/healthyheart.pdf', 'page': 4}, page_content='iv\nTaking Charge:  An Action Plan for Heart Health . . . . . . . . . . . . . . . . . . . 45\nChoose Healthy Foods . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 45Getting Extra Support . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 46\nBlood Pressure and the Dash Eating Plan . . . . . . . . . . . . . . . . . . . . . . . . . 48\nWhat Else Affects Blood Pressure? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 53High Blood Cholesterol and the TLC Program . . . . . . . . . . . . . . . . . . . . . . . . 53Now You’re Cooking: Limiting Saturated Fat,\nTrans Fat, and Cholesterol . . . . . . 53\nMeat, Poultry, and Fish . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 53Milk Products and Eggs . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 54Grains and Grain

Chunking

In [ ]:
# Deviting total text into small part

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [ ]:
len(chunks)

747

In [ ]:
chunks[700]

Document(metadata={'source': '/content/drive/MyDrive/Data/healthyheart.pdf', 'page': 86}, page_content='U.S. Food and Drug Administration for the prevention of heartattacks in those who have never had a heart attack or stroke. \nHowever, a recent, large study has found that among healthy')

Embeddings Creations

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "Enter your HuggingFace token: "

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

<ipython-input-11-0ff5cf6e8b20>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (h

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector Store Creation

In [ ]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
quary = "who is at risk of heart disease?"
search_results= vectorstore.similarity_search(quary)

In [ ]:
search_results

[Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/Data/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/Data/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/drive/MyDrive/Data/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chances of developingheart disease, including its many'),
 Document(me

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k':5})

In [ ]:
retriever.get_relevant_documents(quary)

<ipython-input-16-d1abac5ab37c>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents(quary)


[Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/Data/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/Data/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/drive/MyDrive/Data/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chances of developingheart disease, including its many'),
 Document(me

LLM Model loading


In [ ]:
llm = LlamaCpp(
    model_path="/content/drive/MyDrive/Data_Gen_Guvi/BioMistral-7B.Q4_K_M.gguf",
    temperature=0.2,
    max_tokens = 2048,
    top_p=1
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/Data_Gen_Guvi/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.at

Use LLM and retriver and query, to generate final response ,[use prompt template]

In [ ]:

template = """
<|context|>
You are an Medical Assistant that follows the instructions and generate the accurate response based on the query and the context provided.
Please be truthful and give direct answers.

<|user|>
{query}

<|assistant|>
"""


In [ ]:

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate


In [ ]:

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response=rag_chain.invoke(quary)


llama_print_timings:        load time =    6152.49 ms
llama_print_timings:      sample time =      32.41 ms /    49 runs   (    0.66 ms per token,  1511.74 tokens per second)
llama_print_timings: prompt eval time =   37241.57 ms /    69 tokens (  539.73 ms per token,     1.85 tokens per second)
llama_print_timings:        eval time =   35921.24 ms /    48 runs   (  748.36 ms per token,     1.34 tokens per second)
llama_print_timings:       total time =   73254.31 ms /   117 tokens


In [ ]:
response

'The people who have a family history of heart disease, are overweight or obese, do not exercise regularly, smoke, drink alcohol, have high blood pressure, high cholesterol, or diabetes are at risk for heart disease.'

In [ ]:

import sys

while True:
  user_input = input(f"Input quary: ")
  if user_input == 'exit':
    print("Exiting...")
    sys.exit()
  if user_input=="":
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ", result)

Llama.generate: 52 prefix-match hit, remaining 18 prompt tokens to eval

llama_print_timings:        load time =    6152.49 ms
llama_print_timings:      sample time =      18.52 ms /    29 runs   (    0.64 ms per token,  1566.30 tokens per second)
llama_print_timings: prompt eval time =   10674.42 ms /    18 tokens (  593.02 ms per token,     1.69 tokens per second)
llama_print_timings:        eval time =   20275.97 ms /    28 runs   (  724.14 ms per token,     1.38 tokens per second)
llama_print_timings:       total time =   31010.19 ms /    46 tokens


Answer:  The age group most affected by heart disease is 65 years and older, followed by 45 to 64 years old.


Llama.generate: 53 prefix-match hit, remaining 21 prompt tokens to eval

llama_print_timings:        load time =    6152.49 ms
llama_print_timings:      sample time =      39.62 ms /    60 runs   (    0.66 ms per token,  1514.42 tokens per second)
llama_print_timings: prompt eval time =   11717.87 ms /    21 tokens (  557.99 ms per token,     1.79 tokens per second)
llama_print_timings:        eval time =   43420.22 ms /    59 runs   (  735.94 ms per token,     1.36 tokens per second)
llama_print_timings:       total time =   55235.66 ms /    80 tokens


Answer:  The foods that can help reduce the risk of heart disease include fruits, vegetables, whole grains, lean proteins, low-fat dairy products, and healthy oils. It is also important to limit saturated and trans fats, added sugars, and sodium.
